# Stable Diffusion notebook by [@alexpopo](https://github.com/alexpopo/alex_stable_diffusion_webui)

This let's you generate images with CompVis/Stability [Stable Diffusion](https://github.com/CompVis/stable-diffusion) with bonus [KLMS sampling](https://github.com/crowsonkb/k-diffusion.git) from [@RiversHaveWings](https://twitter.com/RiversHaveWings)

You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/runwayml/stable-diffusion-v1-5).

Other Stable Diffusion tools:
* [Deforum Stable Diffusion](https://colab.research.google.com/github/deforum/stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb) by [deforum_art](https://twitter.com/deforum_art) team
* [HuggingFace space for Stable Diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
* [Stable Diffusion Interpolation](https://colab.research.google.com/drive/1EHZtFjQoRr-bns1It5mTcOVyZzZD9bBc?usp=sharing) by [@ygantigravity](https://twitter.com/ygantigravity)
* [Stable Diffusion with diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=zHkHsdtnry57)
* [Prompt-space Interpolation](https://colab.research.google.com/drive/1gj8_EPyntZuaiQuzYOzog5YKA74esoB3#scrollTo=3g5rSm43STsW) by [@DigThatData](https://twitter.com/DigThatData)
* [Stable Diffusion Lite](https://colab.research.google.com/drive/1cl3d84B5AXepycKAfCba77faUFBRGY2O?usp=sharing#scrollTo=Rn06QzsdsICR) by [@future__art](https://twitter.com/future__art)
* [Stable Diffusion web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)
* More to come!!

If you're looking for more Ai art tools check here [Ai generative art tools list](https://pharmapsychotic.com/tools.html).

if you encounter any issues, feel free to discuss them.[Discord Support](https://discord.com/channels/1002292111942635562)

In [ ]:
#@title Check GPU
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Check Environment
#@markdown ##### 谷歌云端硬盘：仅第一次需要下载资料，之后持久保存在谷歌云端硬盘
#@markdown ##### 简单便捷方式：每次重新连接CoLab需要重新下载所有资料，不需要云硬盘

import os
import time

if os.path.exists("/content/gdrive/MyDrive/"):
  print('Gdrive connected, using Gdrive storage ...')
  print("仅第一次需要下载资料，之后持久保存在谷歌云端硬盘")
else:
  print('[1;31mGdrive not connected, using colab storage ...')
  print("每次重新连接CoLab需要重新下载所有资料，云硬盘不是必须的")

Gdrive not connected, using colab storage ...
每次重新连接CoLab需要重新下载所有资料，云硬盘不是必须的


In [ ]:
#@title First Base Requirements

# 在尝试安装之前，请确保满足所有必需的依赖项
print("Check Common Software Versions ...")
!git --version
print()
print("git-v2.12及之后版本集成了git-lfs")
print("-"*80)
!apt --version
print("-"*80)
!python3 --version
print("-"*80)
!wget --version | grep "^GNU Wget"
print("-"*80)

print('Installing base requirements...')

# Debian-based:
!sudo apt install python3-venv

Check Common Software Versions ...
git version 2.25.1

git-v2.12及之后版本集成了git-lfs
--------------------------------------------------------------------------------
apt 2.0.9 (amd64)
--------------------------------------------------------------------------------
Python 3.8.10
--------------------------------------------------------------------------------
GNU Wget 1.20.3 built on linux-gnu.
--------------------------------------------------------------------------------
Installing base requirements...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  python-pip-whl python3.8-venv
The following NEW packages will be installed:
  python-pip-whl python3-venv python3.8-venv
0 upgraded, 3 newly installed, 0 to remove and 28 not upgraded.
Need to get 1,812 kB

In [ ]:
# python -m pip install -U -I --no-deps xformers-0.0.14.dev0-cp310-cp310-win_amd64.whl

test_outputs_path = "/content/TestSDWebUI"
%mkdir -p $test_outputs_path
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git $test_outputs_path/stable-diffusion-webui

# !pip3 list | grep "pip"
!pip3 list | grep -e "xformers" -e "scipy"
print("-"*80)

%cd /content/TestSDWebUI/stable-diffusion-webui
!python3 -m venv tutorial-env
!source tutorial-env/bin/activate; pip3 install --no-deps xformers==0.0.16rc425; pip3 list

Cloning into '/content/TestSDWebUI/stable-diffusion-webui'...
remote: Enumerating objects: 16079, done.
remote: Total 16079 (delta 0), reused 0 (delta 0), pack-reused 16079
Receiving objects: 100% (16079/16079), 27.20 MiB | 26.78 MiB/s, done.
Resolving deltas: 100% (11292/11292), done.
scipy                         1.7.3
--------------------------------------------------------------------------------
/content/TestSDWebUI/stable-diffusion-webui
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50.9 MB 84 kB/s 
Package       Version    
------------- -----------
pip           20.0.2     
pkg-resources 0.0.0      
setuptools    44.0.0     
xformers      0.0.16rc425


In [ ]:
import sys
import xformers

print(sys.path)

ModuleNotFoundError: ignored

In [ ]:
!pip3 list | grep -e "xformers" -e "scipy"

scipy                         1.7.3
